# 1、多输入多输出通道

实现多输入通道互相关运算

In [1]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X,K):       #X,K 为3d的 第一维是通道数  
    return sum(d2l.corr2d(x,k) for x , k in zip (X ,K))    #对每一个通道数变量，取小矩阵

验证互相关运算的输出

In [3]:
X = torch.tensor([[[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]],
                  [[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]]
                 ])
K = torch.tensor([[[0.0,1.0],[2.0,3.0]],[[1.0,2.0],[3.0,4.0]]])

corr2d_multi_in(X,K)


tensor([[ 56.,  72.],
        [104., 120.]])

## 1.2、计算多个通道的输出的互相关函数

In [4]:
def corr2d_multi_in_out(X,K):  #X输入为3d, K为4d ；【个数(输出通道)，通道数，高，宽】
    return torch.stack([corr2d_multi_in(X,k) for k in K],0) #stack是一个个堆叠操作
K = torch.stack((K,K+1,K+2),0)  #三个卷积核堆叠
K.shape

torch.Size([3, 2, 2, 2])

In [5]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

# 2、1 * 1卷积

1 * 1 卷积等效于全连接层

In [10]:
def corr2d_multi_in_out_1x1(X,K):           #1 *1 的多输入多输出的互相关的操作
    c_i,h ,w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i,h*w))             #将高，宽拉成一条0为向量
    K = K.reshape((c_o,c_i))
    Y = torch.matmul(K,X)
    return Y.reshape((c_o,h,w))           # c_0不变，后面一维向量再reshape回h*w矩阵

X = torch.normal(0,1,(3,3,3))            #输入为3*3*3
K = torch.normal(0,1,(2,3,1,1))          #核 2*3*1*1

Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X,K)
assert float(torch.abs(Y1-Y2).sum()) <1e-6